<a href="https://colab.research.google.com/github/sidfeiner/EpitopeWorkshop/blob/master/EpitopeWorkshop-Sid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from getpass import getpass
token = getpass('insert token')
!git clone https://$token@github.com/sidfeiner/EpitopeWorkshop.git


insert token··········
Cloning into 'EpitopeWorkshop'...
remote: Enumerating objects: 571, done.
remote: Counting objects: 100% (571/571), done.
remote: Compressing objects: 100% (321/321), done.
remote: Total 571 (delta 362), reused 447 (delta 246), pack-reused 0
Receiving objects: 100% (571/571), 15.59 MiB | 21.09 MiB/s, done.
Resolving deltas: 100% (362/362), done.


In [2]:
!rm -fr workshop/
!mv EpitopeWorkshop workshop

In [3]:
!cd workshop/ && pip3 install -e .
!pip install fsspec # For dask

Obtaining file:///content/workshop
     |████████████████████████████████| 2.3 MB 5.2 MB/s 
     |████████████████████████████████| 135 kB 30.9 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
     |████████████████████████████████| 41 kB 192 kB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=f5d2cb98736579d5279b9f4c51995ec5ab97db393e927b83d06fab11742a667c
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50466 sha256=3a01d9c6e77645210abf065c25f420ea2376e7522fcb061954c6ab361458ee26
  Stored in directory: /root/.cache/pip/wheels/38/f7/65/161985e7311dd484a23b3a5c9149995dbf11db6cede602e7ef
  Created wheel for quantiprot: filename=quantiprot-0.2.4-py3-none-any.whl size=151137 sha256=0b636f7269926ff08bde6ad0101d04b90d139501b7ad45c79220210a851535fb
  Stored in directory: /root/.cache/pip/wheels/93/8d/6

In [4]:
import sys
import os
from google.colab import drive

WORKSHOP_ROOT = '/content/workshop/'
DRIVE_MOUNT = '/content/drive'
DRIVE_ROOT = os.path.join(DRIVE_MOUNT, 'My Drive', 'Education',   'TAU', 'year-3', 'semester-b', 'workshop')

sys.path.append(WORKSHOP_ROOT)
!mkdir -p {DRIVE_MOUNT}
drive.mount(DRIVE_MOUNT)

Mounted at /content/drive


In [5]:
import dask as dd
import pandas as pd
import numpy as np

print(pd.__version__, dd.__version__, np.__version__)

1.1.5 2.12.0 1.19.5


In [6]:
import pandas as pd 

def print_balanced_data(df: pd.DataFrame):
    vals = df[contract.IS_IN_EPITOPE_COL_NAME].value_counts(normalize=True,
                                                            sort=True)
    print(f"total records: {len(df)}. Frequencies:")
    for val, freq in enumerate(vals):
        print(f"{val}: {freq * 100}%")

In [7]:
# Configurations

sequences_files_dir = '/content/workshop/data/iedb-linear-epitopes-parts'
window_size = 9
partitions_amt=4
limit_sequences_amt = None
batch_size = 20
oversampling_altercation_pct_min = 98
oversampling_altercation_pct_max = 102
oversampling_change_val_proba = 0.2

In [8]:
from typing import Optional
from EpitopeWorkshop.process.features import FeatureCalculator
from EpitopeWorkshop.process.balance.balance import OverSamplingBalancer
from EpitopeWorkshop.process.balance.transform import FeatureTransformer
import pickle
import shutil
from EpitopeWorkshop.common import contract


def process_file(sequences_file_path: str, partitions_amt: int,
                 with_sliding_window: bool,
                 window_size: int, limit_sequences_amt: Optional[int],
                 oversampling_change_val_proba: float,
                 oversampling_altercation_pct_min: int,
                 oversampling_altercation_pct_max: int):
  logging.info(f"handling file {sequences_file_path}")
  original_dir_path, basename = os.path.split(sequences_file_path)
  name, ext = os.path.splitext(basename)
  final_dir = os.path.join(DRIVE_ROOT, 'features')
  done_dir = os.path.join(original_dir_path, 'handled')
  os.makedirs(final_dir, exist_ok=True)
  os.makedirs(done_dir, exist_ok=True)

  df = read.load_fasta_row_per_window(sequences_file_path, with_sliding_window, window_size, limit_sequences_amt)

  # under_balancer = UnderSamplingBalancer(
  #     contract.IS_IN_EPITOPE_COL_NAME,
  #     balances={
  #         0: 0.7,
  #         1: 0.3
  #     }
  # )
  # df = under_balancer.balance(df)
  # print_balanced_data(df)

  ddf = dd.from_pandas(df, npartitions=partitions_amt)
  calculator = FeatureCalculator()

  logging.info("calculating features")
  df = calculator.calculate_features(ddf)

  logging.info("saving full file to pickle file")
  raw_data_path = os.path.join(final_dir, f"{basename}_features{ext}")
  df.to_pickle(
      path=raw_data_path,
      protocol=pickle.HIGHEST_PROTOCOL
  )

  feature_transformer = FeatureTransformer(
      oversampling_altercation_pct_min,
      oversampling_altercation_pct_max,
      oversampling_change_val_proba
  )
  balancer = OverSamplingBalancer(
      contract.IS_IN_EPITOPE_COL_NAME,
      [(contract.CALCULATED_FEATURES_COL_NAME, feature_transformer.transform)],
      balances={
          0: 0.5,
          1: 0.5
      }
  )

  logging.info("balancing by over sampling")

  df = balancer.balance(df)
  print_balanced_data(df)

  logging.info("saving balanced to pickle file")
  raw_data_path = os.path.join(final_dir, f"{basename}_features_balanced{ext}")
  df.to_pickle(
      path=raw_data_path,
      protocol=pickle.HIGHEST_PROTOCOL
  )

  shutil.move(sequences_file_path, done_dir)
  logging.info(f"done handling {sequences_file_path}")

In [9]:
from EpitopeWorkshop.process import read
import dask.dataframe as dd
import logging
import glob 

log_format = "%(asctime)s : %(threadName)s: %(levelname)s : %(name)s : %(module)s : %(message)s"
logging.basicConfig(format=log_format, level=logging.DEBUG)

files = glob.glob(os.path.join(sequences_files_dir, '*.fasta'))
for file in files:
    process_file(
        file, partitions_amt, True, window_size, limit_sequences_amt,
        oversampling_change_val_proba, oversampling_altercation_pct_min, oversampling_altercation_pct_max
    )

df = read.load_fasta_row_per_window(sequences_file_path, True, window_size, limit_sequences_amt)
ddf = dd.from_pandas(df, npartitions=partitions_amt)

2021-08-31 09:23:23,628 : MainThread: INFO : root : <ipython-input-8-cf0f77590c97> : handling file /content/workshop/data/iedb-linear-epitopes-parts/iedb_linear_epitopes_67.fasta
2021-08-31 09:25:31,833 : MainThread: INFO : root : <ipython-input-8-cf0f77590c97> : calculating features
2021-08-31 09:44:03,208 : MainThread: INFO : root : <ipython-input-8-cf0f77590c97> : saving full file to pickle file
2021-08-31 09:44:21,395 : MainThread: INFO : root : <ipython-input-8-cf0f77590c97> : balancing by over sampling


ValueError: ignored

In [17]:
from EpitopeWorkshop.dataset.EpitopeDataset import EpitopeDataset

calculated_features = df[contract.CALCULATED_FEATURES_COL_NAME]
labels = df[contract.IS_IN_EPITOPE_COL_NAME]
ds = EpitopeDataset(calculated_features, labels)
ds_train, ds_valid, ds_test = ds.splits()
print(f"train size: {len(ds_train)}, valid size: {len(ds_valid)}, test size: {len(ds_test)}")

dl_train, dl_valid, dl_test = ds.iters(batch_size=batch_size)


train size: 763, valid size: 116, test size: 206


In [20]:
from EpitopeWorkshop.cnn.cnn import CNN
from EpitopeWorkshop.cnn.train import train_model

cn = CNN()
train_model(cn, dl_train=dl_train)

Streaming output truncated to the last 5000 lines.
          ...,
          [ 2.7800, -3.5000,  1.2830,  ...,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  1.8000,  0.8150,  ...,  0.0000,  0.0000,  0.0000],
          [ 3.9500, -3.5000,  1.3480,  ...,  0.0000,  0.0000,  0.0000]]],


        [[[ 2.7200, -1.6000,  1.2360,  ...,  0.0000,  0.0000,  0.0000],
          [ 4.7700, -3.9000,  1.5450,  ...,  0.0000,  0.0000,  0.0000],
          [ 4.6600, -3.2000,  1.1800,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 1.6000, -0.8000,  1.1150,  ...,  0.0000,  0.0000,  0.0000],
          [ 4.4300,  1.9000,  0.7140,  ...,  0.0000,  0.0000,  0.0000],
          [ 2.7200, -1.6000,  1.2360,  ...,  0.0000,  0.0000,  0.0000]]],


        [[[ 4.0800,  4.5000,  0.5970,  ...,  0.0000,  0.0000,  0.0000],
          [ 3.0000,  4.2000,  0.6060,  ...,  0.0000,  0.0000,  0.0000],
          [ 2.6260, -0.6999,  1.2198,  ...,  0.0000,  0.0000,  0.0000],
          ...,
          [ 2.7200, -1.6000,  1.

In [ ]:
from torchtext import data
BATCH_SIZE = 25
dl_train, dl_valid, dl_test = ds.iters(batch_size=32)

x0, y0 = ds[0]

In [ ]:
from EpitopeWorkshop.cnn.cnn import CNN

model = CNN(len(contract.NETWORK_INPUT_ARGS))

print('x0 shape=', x0.shape, end='\n\n')
print('LeNet(x0)=', model(x0), end='\n\n')
print('shape=', net(x0).shape)

x0 shape= torch.Size([26])



RuntimeError: ignored